In [5]:
import os
import librosa
import numpy as np
import pandas as pd

def preprocess_and_save_audio(folder):
    # Directory where processed files will be saved
    processed_dir = os.path.join(os.getcwd(), 'processed-audio')
    
    # Check if the directory exists, if not, create it
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)

    file_path = os.path.join("E:\\archive\\train", folder, "audio.opus")
    
    # Load the audio file
    try:
        y, sr = librosa.load(file_path, sr=None)
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfcc


mfcc = preprocess_and_save_audio("0297cae08327285a8abb3926aa4c1c9d")

mfcc.size



270205

In [ ]:
136,296,35353,2,0,L|104:300,6,19.1249994163513,2|0|0|0|0|0|0,1:2|1:0|1:0|1:0|1:0|1:0|1:0,0:0:0:0:

136,296,35353,2,0,L|something,19.1249994163513,2,1:2,0:0:0:0:,1
104,300,newtime,2,0,L|something:300,6,19.1249994163513,0,1:0,0:0:0:0:,0
104,300,newtime,2,0,L|104:something,6,19.1249994163513,0,1:0,0:0:0:0:,0


In [66]:
import math
from typing import List, Tuple

# # Function to calculate the slider velocity
# def calculate_slider_velocity(SliderMultiplier, bpm):
#     # Base velocity in osu! is 100 pixels per beat
#     base_velocity = SliderMultiplier * 100
    
#     # Convert bpm to beats per second
#     beats_per_second = bpm / 60.0
    
#     # Calculate velocity in pixels per second
#     velocity_per_second = base_velocity * beats_per_second
    
#     return velocity_per_second

# Function to interpolate linear points
def interpolate_linear(start, end, t):
    return start + t * (end - start)

# Function to calculate a point on a quadratic Bezier curve
def bezier_point(t, points):
    n = len(points) - 1
    x = sum(math.comb(n, i) * (1 - t)**(n - i) * t**i * points[i][0] for i in range(n + 1))
    y = sum(math.comb(n, i) * (1 - t)**(n - i) * t**i * points[i][1] for i in range(n + 1))
    return x, y

# Function to split slider into segments
def split_slider(slider_data):
    # Parse slider_data
    parts = slider_data.split(',')
    print(parts)
    start_point = tuple(map(int, parts[:2]))
    slider_type = parts[5][0]
    path_data = parts[5][2:]
    slider_points = parts[8].split('|')
    slider_points_data = parts[9].split('|')
    num_points = len(slider_points)
    # print(path_data)
    path_points = [start_point] + [tuple(map(int, p.split(':'))) for p in path_data.split('|')]
    print(slider_points_data)
   

    # # Calculate slider velocity
    # velocity = calculate_slider_velocity(SliderMultiplier,bpm)
    # duration = velocity * float(parts[7])
    # print(duration)
    
    # Total time for slider
    # duration = float(parts[7])
    
    # Time increment per segment
    # time_increment = duration / num_points

    # Initialize new segments list
    new_segments = []
    
    print(num_points)
    # Generate new segments based on slider type
    if slider_type == 'L':
        # Linear interpolation
        for i in range(num_points):
            t = i / (num_points - 1)
            new_x = interpolate_linear(start_point[0], path_points[-1][0], t)
            new_y = interpolate_linear(start_point[1], path_points[-1][1], t)
            segment = f"{int(new_x)},{int(new_y)},{int(parts[2])},{parts[3]},{int(parts[4])},L|{int(new_x)}:{int(new_y)},{int(parts[6])},{slider_points[i]},{slider_points_data[i]},{parts[-1]},{0 if i == 0 else 2 if i == num_points - 1 else 1}"
            new_segments.append(segment)

    elif slider_type == 'B':
        # Bezier curve interpolation
        for i in range(num_points):
            t = i / (num_points - 1)
            new_x, new_y = bezier_point(t, path_points)
            segment = f"{int(new_x)},{int(new_y)},{int(parts[2])},{parts[3]},{int(parts[4])},B|{int(new_x)}:{int(new_y)},{int(parts[6])},{slider_points[i]},{slider_points_data[i]},{parts[-1]},{0 if i == 0 else 2 if i == num_points - 1 else 1}"
            new_segments.append(segment)

    # Similar handling for other types ('C', 'P')...
    # print(len(new_segments))

    return new_segments

# Example slider data and split operation
slider_data = "144,136,155821,2,0,B|160:140|160:140|180:132|180:132|200:140|200:140|220:132|220:132|236:140|236:140|256:132|256:132|276:140|276:140|292:132|292:132|312:140|312:140|332:132|332:132|352:140|352:140|368:136,1,240,10|0,0:0|0:0,0:0:0:0:"
# SliderMultiplier = 1.7
# bpm = 175.0

# Split the slider into segments
new_slider_segments = split_slider(slider_data)
# Output the new segments
for segment in new_slider_segments:
    print(segment)


['144', '136', '155821', '2', '0', 'B|160:140|160:140|180:132|180:132|200:140|200:140|220:132|220:132|236:140|236:140|256:132|256:132|276:140|276:140|292:132|292:132|312:140|312:140|332:132|332:132|352:140|352:140|368:136', '1', '240', '10|0', '0:0|0:0', '0:0:0:0:']
['0:0', '0:0']
2
144,136,155821,2,0,B|144:136,1,10,0:0,0:0:0:0:,0
368,136,155821,2,0,B|368:136,1,0,0:0,0:0:0:0:,2


In [68]:
import math
from typing import List, Tuple

# Function to interpolate linear points
def interpolate_linear(start, end, t):
    return start + t * (end - start)

# Function to calculate a point on a quadratic Bezier curve
def bezier_point(t, points):
    n = len(points) - 1
    x = sum(math.comb(n, i) * (1 - t)**(n - i) * t**i * points[i][0] for i in range(n + 1))
    y = sum(math.comb(n, i) * (1 - t)**(n - i) * t**i * points[i][1] for i in range(n + 1))
    return x, y

# Function to calculate a point on a perfect circle (P slider type)
def circle_point(center, radius, angle):
    return (
        center[0] + radius * math.cos(angle),
        center[1] + radius * math.sin(angle)
    )

# Function to split slider into segments
def split_slider(slider_data: str):
    # Parse slider_data
    parts = slider_data.split(',')
    start_point = tuple(map(int, parts[:2]))
    slider_type = parts[5][0]
    path_data = parts[5][2:]
    slider_points = parts[8].split('|')
    slider_points_data = parts[9].split('|')
    num_points = len(slider_points)

    # Calculate path points
    path_points = [start_point] + [tuple(map(int, p.split(':'))) for p in path_data.split('|')]

    # Initialize new segments list
    new_segments = []
    
    # Generate new segments based on slider type
    if slider_type == 'L':
        # Linear interpolation
        for i in range(num_points):
            t = i / (num_points - 1)
            new_x = interpolate_linear(start_point[0], path_points[-1][0], t)
            new_y = interpolate_linear(start_point[1], path_points[-1][1], t)
            segment = f"{int(new_x)},{int(new_y)},{int(parts[2])},{parts[3]},{int(parts[4])},L|{int(new_x)}:{int(new_y)},{int(parts[6])},{slider_points[i]},{slider_points_data[i]},{parts[-1]},{0 if i == 0 else 2 if i == num_points - 1 else 1}"
            new_segments.append(segment)

    elif slider_type == 'B':
        # Bezier curve interpolation
        for i in range(num_points):
            t = i / (num_points - 1)
            new_x, new_y = bezier_point(t, path_points)
            segment = f"{int(new_x)},{int(new_y)},{int(parts[2])},{parts[3]},{int(parts[4])},B|{int(new_x)}:{int(new_y)},{int(parts[6])},{slider_points[i]},{slider_points_data[i]},{parts[-1]},{0 if i == 0 else 2 if i == num_points - 1 else 1}"
            new_segments.append(segment)

    elif slider_type == 'P':
        # Perfect circle interpolation
        # Extract center and radius
        center = path_points[0]
        radius = math.sqrt((path_points[1][0] - center[0])**2 + (path_points[1][1] - center[1])**2)
        total_points = num_points
        angle_increment = 2 * math.pi / total_points
        
        for i in range(num_points):
            angle = i * angle_increment
            new_x, new_y = circle_point(center, radius, angle)
            new_time = int(parts[2])  # This should be adjusted based on actual time increments
            segment = f"{int(new_x)},{int(new_y)},{new_time},{parts[3]},{int(parts[4])},P|{int(new_x)}:{int(new_y)},{int(parts[6])},{slider_points[i]},{slider_points_data[i]},{parts[-1]},{0 if i == 0 else 2 if i == num_points - 1 else 1}"
            new_segments.append(segment)

    return new_segments

# Example slider data and split operation
slider_data = "431,86,96039,6,0,P|406:144|433:238,1,157.500006008148,0|0,1:0|0:0,0:0:0:0:"

# Split the slider into segments
new_slider_segments = split_slider(slider_data)
# Output the new segments
for segment in new_slider_segments:
    print(segment)


494,86,96039,6,0,P|494:86,1,0,1:0,0:0:0:0:,0
367,86,96039,6,0,P|367:86,1,0,0:0,0:0:0:0:,2


In [32]:
sr = 20000
audio_length_sec = 1
frame_size = 2000
hop_length = 500
n_mfcc = 13

total_samples = sr * audio_length_sec
num_frames = (total_samples - frame_size) // hop_length + 1
mfcc_dim = (n_mfcc, num_frames)

print(total_samples)
# print(22050/100)
print(num_frames)
# print((4303/1000)*100)
print(mfcc_dim)

20000
37
(13, 37)


In [32]:
sm=2.1
sv = 200
svm=100/sv
beatlen = 60000/175
print(beatlen)
# svm = 1
length = 420
# esv = svm * sm
# -50 x = 2
# x=2/-50=4/-100=-0.
# 100/25

t = 148153
m = t//60000
s = (t - (m*60000))
ms = s%1000
s=s//1000
print(f"{m}:{s}:{ms}")
m=2
s=29
ms=524
timediff = ( m*60000 + s*1000 + ms) - t
print(timediff)
print(length / (sm * 100 * svm) * beatlen)

342.85714285714283
2:28:153
1371
1371.4285714285713


In [72]:
sm=2.1
sv = 200
svm=100/sv
beatlen = 60000/175
# print(beatlen)

length = 52.5
# esv = svm * sm
# -50 x = 2
# x=2/-50=4/-100=-0.
# 100/25

t = 154668
m = t//60000
s = (t - (m*60000))
ms = s%1000
s=s//1000
print(f"{m}:{s}:{ms}")
m=2
s=34
ms=839
timediff = ( m*60000 + s*1000 + ms) - t
print(timediff)


calctime = length / (sm * 100 * svm) * beatlen
print(calctime*1)

2:34:668
171
171.42857142857142


# if timing point not there then timingpoint above one is used as same for uninherited also

In [45]:
sm=2.1
sv = 200
svm=100/sv
beatlen = 60000/175
print(beatlen)
# svm = 1
length = 420
# esv = svm * sm
# -50 x = 2
# x=2/-50=4/-100=-0.
# 100/25

t = 45486
m = t//60000
s = (t - (m*60000))
ms = s%1000
s=s//1000
print(f"{m}:{s}:{ms}")
m=2
s=29
ms=524
timediff = ( m*60000 + s*1000 + ms) - t
print(timediff)
print(length / (sm * 100 * svm) * beatlen)

342.85714285714283
0:45:486
104038
1371.4285714285713


In [37]:
hit_type = 12
# Convert hit_type to a binary string
binary_representation = bin(hit_type)[-4:]

# Check the last three bits
last_three_bits = binary_representation.zfill(3)  # Ensure it's always 3 bits long

# Find indices of '1' bits
indices_of_ones = [i for i, bit in enumerate(reversed(last_three_bits)) if bit == '1']

print(f"Binary representation: {last_three_bits}")
print(f"Indices with '1': {indices_of_ones}")
print( 3 in indices_of_ones)

Binary representation: 1100
Indices with '1': [2, 3]
True


In [43]:
subpart = [0] 
if len(subpart) < 4:
        subpart = subpart + [0]*(4-len(subpart))
print(subpart)

[0, 0, 0, 0]
